In [ ]:
import pandas as pd
import math
import numpy as np

pd.set_option('display.max_columns', None) 
mWindow = 10 #in sec, but should eventually be in ms
fWindow = 2 #in sec, but should eventually be in ms

In [ ]:
rawData = pd.read_csv('./raw-data.csv')
fullData = pd.DataFrame(columns=['Timestamp','m1','m2','m3','m4'])

In [ ]:
rawData.head(n=11)

In [ ]:
i=0
for row in rawData.itertuples():
    m1 = math.sqrt(row[2]**2 + row[3]**2 + row[4]**2)
    m2 = math.sqrt(row[5]**2 + row[6]**2 + row[7]**2)
    m3 = math.sqrt(row[8]**2 + row[9]**2 + row[10]**2)
    m4 = math.sqrt(row[11]**2 + row[12]**2 + row[13]**2) 
    rawData.at[i, 'm1'] = m1
    rawData.at[i, 'm2'] = m2
    rawData.at[i, 'm3'] = m3
    rawData.at[i, 'm4'] = m4
    i += 1

In [ ]:
#subtract the mean from the data within a time window centered around the current sample
for row in rawData.itertuples():
    centerTime = row.Timestamp
    i = row.Index
    laterNotDone = True
    beforeNotDone = True
    m1List = [row.m1]
    m2List = [row.m2]
    m3List = [row.m3]
    m4List = [row.m4]
    x = 1
    
    while beforeNotDone and i-x >= 0:
        if centerTime - rawData.loc[i-x].Timestamp <= mWindow/2 :
            m1List.append(rawData.loc[i-x].m1)
            m2List.append(rawData.loc[i-x].m2)
            m3List.append(rawData.loc[i-x].m3)
            m4List.append(rawData.loc[i-x].m4)
            x += 1
        else:
            beforeNotDone = False
            x = 1
            
    while laterNotDone and i + x < len(rawData):
        if rawData.loc[i+x].Timestamp - centerTime <= mWindow/2 :
            m1List.append(rawData.loc[i+x].m1)
            m2List.append(rawData.loc[i+x].m2)
            m3List.append(rawData.loc[i+x].m3)
            m4List.append(rawData.loc[i+x].m4)
            x += 1
        else:
            laterNotDone = False
            x = 1
            
    m1 = row.m1 - np.mean(m1List)
    m2 = row.m2 - np.mean(m2List)
    m3 = row.m3 - np.mean(m3List)
    m4 = row.m4 - np.mean(m4List)
    
    fullData = fullData.append({'Timestamp': row.Timestamp, 'm1': m1, 'm2':m2, 'm3':m3, 'm4':m4}, ignore_index=True)

In [ ]:
#calculate max and prod for each row
for row in fullData.itertuples():  
    i = row.Index
    maxm1m2 = max(row.m1, row.m2)
    maxm3m4 = max(row.m3, row.m4)
    maxm1m3 = max(row.m1, row.m3)
    maxm2m4 = max(row.m2, row.m4)
    maxall = max(row.m1, row.m2, row.m3, row.m4)
    prodm1m2 = row.m1 * row.m2
    prodm3m4 = row.m3 * row.m4
    prodm1m3 = row.m1 * row.m3
    prodm2m4 = row.m2 * row.m4
    prodall = row.m1 * row.m2 * row.m3 * row.m4
    fullData.at[i, 'maxm1m2'] = maxm1m2
    fullData.at[i, 'maxm3m4'] = maxm3m4
    fullData.at[i, 'maxm1m3'] = maxm1m3
    fullData.at[i, 'maxm2m4'] = maxm2m4
    fullData.at[i, 'maxall'] = maxall
    fullData.at[i, 'prodm1m2'] = prodm1m2
    fullData.at[i, 'prodm3m4'] = prodm3m4
    fullData.at[i, 'prodm1m3'] = prodm1m2
    fullData.at[i, 'prodm2m4'] = prodm3m4
    fullData.at[i, 'prodall'] = prodall

In [ ]:
#calculate the mean, max, min, standard deviation, and z-score 
#from data within a time window centered around the current sample
for row in fullData.itertuples(): 
    i = row.Index
    centerTime = row.Timestamp
    laterNotDone = True
    beforeNotDone = True
    x = 1
    
    m1List = [row.m1]
    m2List = [row.m2]
    m3List = [row.m3]
    m4List = [row.m4]
    maxm1m2List = [row.maxm1m2]
    maxm3m4List = [row.maxm3m4]
    maxm1m3List = [row.maxm1m3]
    maxm2m4List = [row.maxm2m4]
    maxallList = [row.maxall]
    prodm1m2List = [row.prodm1m2]
    prodm3m4List = [row.prodm3m4]
    prodm1m3List = [row.prodm1m3]
    prodm2m4List = [row.prodm2m4]
    prodallList = [row.prodall]
    featureNames = ['m1','m2','m3','m4','maxm1m2','maxm3m4','maxm1m3','maxm2m4',
                    'maxall','prodm1m2','prodm3m4','prodm1m3','prodm2m4','prodall'] 
    featureList = [m1List,m2List,m3List,m4List,maxm1m2List,maxm3m4List,maxm1m3List,maxm2m4List,maxallList,
                   prodm1m2List,prodm3m4List,prodm1m3List,prodm2m4List,prodallList] 
    
    while beforeNotDone and i - x  >= 0:
        if centerTime - fullData.loc[i-x].Timestamp <= fWindow/2 :
            nameIndex = 0
            for ftList in featureList:
                ft = featureNames[nameIndex]
                ftList.append(fullData.loc[i-x][ft])
                nameIndex += 1
            x += 1
        else:
            beforeNotDone = False
            x = 1
            
    while laterNotDone and i + x < len(fullData):
        if fullData.loc[i+x].Timestamp - centerTime <= fWindow/2 :
            nameIndex = 0
            for ftList in featureList:
                ft = str(featureNames[nameIndex])
                ftList.append(fullData.loc[i+x][ft])
                nameIndex += 1
            x += 1
        else:
            laterNotDone = False
            x = 1
            
    nameIndex = 0        
    for ftList in featureList:
        ft = featureNames[nameIndex]
        std = np.std(ftList)
        mean = np.mean(ftList)
        meanft = 'mean_' + ft
        maxft = 'max_' + ft
        minft = 'min_' + ft
        sdft = 'sd_' + ft
        zft = 'z_' + ft
        fullData.at[i, meanft] = mean
        fullData.at[i, maxft] = max(ftList)
        fullData.at[i, minft] = min(ftList)
        fullData.at[i, sdft] = std
        fullData.at[i, zft] = np.divide(fullData.loc[i][ft] - mean, std, where=std!=0)
        nameIndex += 1

In [ ]:
fullData.head()

In [ ]:
fullData.to_csv('testOutput.csv', index=False)